# Multioutput-Ensemble Combinations for Ordering Process Step

In [1]:
import numpy as np
import csv
import time
import pandas
import signal
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate

In [3]:
from sklearn.multioutput import ClassifierChain
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import MultiOutputClassifier

from sklearn.ensemble.weight_boosting import AdaBoostClassifier
from sklearn.ensemble.weight_boosting import AdaBoostRegressor
from sklearn.ensemble.bagging import BaggingClassifier
from sklearn.ensemble.bagging import BaggingRegressor
from sklearn.ensemble.forest import ExtraTreesClassifier
from sklearn.ensemble.forest import ExtraTreesRegressor
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.ensemble.forest import RandomForestRegressor

from sklearn.linear_model.bayes import ARDRegression
from sklearn.linear_model.bayes import BayesianRidge
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_decomposition.cca_ import CCA
from sklearn.tree.tree import DecisionTreeClassifier
from sklearn.tree.tree import DecisionTreeRegressor
from sklearn.linear_model.coordinate_descent import ElasticNet
from sklearn.tree.tree import ExtraTreeClassifier
from sklearn.tree.tree import ExtraTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.gpc import GaussianProcessClassifier
from sklearn.gaussian_process.gpr import GaussianProcessRegressor
from sklearn.linear_model.huber import HuberRegressor
from sklearn.neighbors.classification import KNeighborsClassifier
from sklearn.neighbors.regression import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.semi_supervised.label_propagation import LabelPropagation
from sklearn.semi_supervised.label_propagation import LabelSpreading
from sklearn.linear_model.least_angle import Lars
from sklearn.linear_model.coordinate_descent import Lasso
from sklearn.linear_model.least_angle import LassoLars
from sklearn.linear_model.least_angle import LassoLarsIC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model.base import LinearRegression
from sklearn.svm.classes import LinearSVC
from sklearn.svm.classes import LinearSVR
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.neural_network.multilayer_perceptron import MLPClassifier
from sklearn.neural_network.multilayer_perceptron import MLPRegressor
from sklearn.linear_model.coordinate_descent import MultiTaskElasticNet
from sklearn.linear_model.coordinate_descent import MultiTaskLasso
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.svm.classes import NuSVC
from sklearn.svm.classes import NuSVR
from sklearn.linear_model.omp import OrthogonalMatchingPursuit
from sklearn.cross_decomposition.pls_ import PLSCanonical
from sklearn.cross_decomposition.pls_ import PLSRegression
from sklearn.linear_model.passive_aggressive import PassiveAggressiveClassifier
from sklearn.linear_model.passive_aggressive import PassiveAggressiveRegressor
from sklearn.linear_model.perceptron import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model.ransac import RANSACRegressor
from sklearn.neighbors.classification import RadiusNeighborsClassifier
from sklearn.neighbors.regression import RadiusNeighborsRegressor
from sklearn.linear_model.ridge import Ridge
from sklearn.linear_model.ridge import RidgeClassifier
from sklearn.linear_model.stochastic_gradient import SGDClassifier
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.svm.classes import SVC
from sklearn.svm.classes import SVR
from sklearn.linear_model.theil_sen import TheilSenRegressor

In [4]:
tfidf = pandas.read_csv('07 NM PS TFIDF.csv', delimiter=',', encoding='latin-1').fillna('')
tfidf[:3]

,_____________________,_____________________ follow,able,acceptable,accessory,accessory require,accidentally,accompany,accord,account,...,wrong ct,wrong info,wrong info mri,wrong patient,wrong pt,wrong set,wrong tattoo,xrt,xrt date,xrt pt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
ohe = pandas.read_csv('07 NM PS OHE.csv', delimiter=',', encoding='latin-1').fillna('')
ohe[:3]

,Treatment delivery,Radiation treatment prescription scheduling,Pre-treatment quality assurance,Post-treatment completion,Interventional procedure for planning and/or delivery,Contouring and planning,Imaging for treatment planning,Patient medical consultation and physician assessment,On-treatment quality assurance
0,1,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0


In [6]:
multioutputs = [ClassifierChain, MultiOutputRegressor, MultiOutputClassifier]
ensembles = [AdaBoostClassifier, AdaBoostRegressor, BaggingClassifier, BaggingRegressor, ExtraTreesClassifier, ExtraTreesRegressor, GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier, RandomForestRegressor]
bases = [ARDRegression, BayesianRidge, BernoulliNB, CCA, DecisionTreeClassifier, DecisionTreeRegressor, ElasticNet, ExtraTreeClassifier, ExtraTreeRegressor, GaussianNB, GaussianProcessClassifier, GaussianProcessRegressor, HuberRegressor, KNeighborsClassifier, KNeighborsRegressor, KernelRidge, LabelPropagation, LabelSpreading, Lars, Lasso, LassoLars, LassoLarsIC, LinearDiscriminantAnalysis, LinearRegression, LinearSVC, LinearSVR, LogisticRegression, MLPClassifier, MLPRegressor, MultiTaskElasticNet, MultiTaskLasso, MultinomialNB, NearestCentroid, NuSVC, NuSVR, OrthogonalMatchingPursuit, PLSCanonical, PLSRegression, PassiveAggressiveClassifier, PassiveAggressiveRegressor, Perceptron, QuadraticDiscriminantAnalysis, RANSACRegressor, RadiusNeighborsClassifier, RadiusNeighborsRegressor, Ridge, RidgeClassifier, SGDClassifier, SGDRegressor, SVC, SVR, TheilSenRegressor]

In [7]:
# This is a custom scoring function that implements the following.
# If a model were to order the labels in a drop down list
# how far down the list does the user need to look down in order to find all the correct labels?
def lowest_correct(trues, preds):
    num_of_options = len(trues)
    drop_down_options = list(reversed(np.argsort(preds)))
    correct_options = [i for i in range(num_of_options) if trues[i]==1]
    return max([drop_down_options.index(correct_option) for correct_option in correct_options]) + 1
def average_lowest_correct(list_of_trues, list_of_preds):
    length = len(list_of_trues)
    return np.mean([lowest_correct(list(list_of_trues.iloc[i]), list(list_of_preds[i])) for i in range(length)])

In [8]:
def deadline(timeout, *args):
    def decorate(f):
        def handler(signum, frame):
            raise Exception

        def new_f(*args):
            signal.signal(signal.SIGALRM, handler)
            signal.alarm(timeout)
            return f(*args)
            signal.alarm(0)

        new_f.__name__ = f.__name__
        return new_f
    return decorate

In [9]:
@deadline(180)
def cv_ensemble(multioutput, ensemble, xs, ys):
    temp = cross_validate(multioutput(ensemble()), xs, ys, return_train_score=True, scoring=make_scorer(average_lowest_correct), n_jobs=-1, cv=2)
    return [multioutput.__name__, ensemble.__name__, None, (np.sum(temp['fit_time'])+ np.sum(temp['score_time'])).round(2), np.mean(temp['test_score']).round(2)]

In [10]:
models = pandas.DataFrame(columns=['Multioutput', 'Ensemble', 'Base', 'Time', 'Score'])
row = 0

past = time.time()
for multioutput in multioutputs:
    for ensemble in ensembles:
        try:
            results = cv_ensemble(multioutput, ensemble, tfidf, ohe)
            print(results[0], results[1], results[2], results[3], results[4])
            for i in range(5):
                models.at[row, models.columns[i]] = results[i]
            row += 1
        except Exception:
            pass

ClassifierChain AdaBoostClassifier None 16.61 2.41
ClassifierChain BaggingClassifier None 23.05 2.35
ClassifierChain ExtraTreesClassifier None 3.16 2.45
ClassifierChain GradientBoostingClassifier None 45.65 2.38
ClassifierChain RandomForestClassifier None 2.68 2.64
MultiOutputRegressor AdaBoostClassifier None 15.5 2.53
MultiOutputRegressor AdaBoostRegressor None 55.28 2.18
MultiOutputRegressor BaggingClassifier None 28.8 2.49
MultiOutputRegressor BaggingRegressor None 26.99 1.87
MultiOutputRegressor ExtraTreesClassifier None 3.07 2.68
MultiOutputRegressor ExtraTreesRegressor None 35.85 1.89
MultiOutputRegressor GradientBoostingClassifier None 47.44 2.56
MultiOutputRegressor GradientBoostingRegressor None 49.75 1.74
MultiOutputRegressor RandomForestClassifier None 2.06 2.67
MultiOutputRegressor RandomForestRegressor None 22.35 1.84
MultiOutputClassifier AdaBoostClassifier None 14.48 2.53
MultiOutputClassifier AdaBoostRegressor None 50.57 2.18
MultiOutputClassifier BaggingClassifier None

In [11]:
models

,Multioutput,Ensemble,Base,Time,Score
0,ClassifierChain,AdaBoostClassifier,None,16.61,2.41
1,ClassifierChain,BaggingClassifier,None,23.05,2.35
2,ClassifierChain,ExtraTreesClassifier,None,3.16,2.45
3,ClassifierChain,GradientBoostingClassifier,None,45.65,2.38
4,ClassifierChain,RandomForestClassifier,None,2.68,2.64
5,MultiOutputRegressor,AdaBoostClassifier,None,15.5,2.53
6,MultiOutputRegressor,AdaBoostRegressor,None,55.28,2.18
7,MultiOutputRegressor,BaggingClassifier,None,28.8,2.49
8,MultiOutputRegressor,BaggingRegressor,None,26.99,1.87
9,MultiOutputRegressor,ExtraTreesClassifier,None,3.07,2.68


In [12]:
models.to_csv('08 PS ME.csv', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)